<a href="https://colab.research.google.com/github/GrLau/Transformers-For-Stock-Price-Prediction/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Set Up

In [ ]:
!pip install yfinance
!pip install tqdm
!pip install -U tensorflow-addons
!pip install -q "tqdm>=4.36.1"

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np



start = datetime.date(1990, 12, 12)
end = datetime.date(2024, 1, 1)
check_date = datetime.date.today()
codelist = ["AAPL"]

stock_df = yf.download(codelist, start=start, end=end)
stock_check_df = yf.download(codelist, start=end, end=check_date)


stock_train = stock_df['Close'].dropna()
stock_test = stock_check_df['Close'].dropna()



stock_np = stock_df.to_numpy()
stock_train_np = stock_train.to_numpy()
stock_test_np = stock_test.to_numpy()

stock_np = stock_np.reshape(-1)
stock_train_np = stock_train_np.reshape(-1)
stock_test_np = stock_test_np.reshape(-1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import tensorflow as tf
import tensorflow_addons as tfa

# Split a univariate sequence into samples
def split_sequence(sequence, n_steps, n_output):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        out_end_ix = end_ix + n_output
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Define input sequence
raw_seq = stock_train_np
# Choose a number of time steps
n_steps = 3
# Set n_output to a smaller value than the length of the sequence minus n_steps
n_output = len(stock_test_np) # For example

if n_output > len(stock_test_np):
    raise ValueError("n_output must be less than or equal to len(stock_test_np) if you want to evaluate performance")


# Split into samples
X, y = split_sequence(raw_seq, n_steps, n_output)

# Reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))


tqdm_callback = tfa.callbacks.TQDMProgressBar()


# Define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mae')

# Fit model
model.fit(X, y, epochs=200, verbose=0, callbacks=[tqdm_callback])


# Demonstrate prediction
x_input = array(stock_train_np[-n_steps:])
x_input = x_input.reshape((1, n_steps, n_features))
predictions = model.predict(x_input, verbose=1)

Epoch 1/200
206/206 [==============================] - 3s 7ms/step - loss: 9.7410
Epoch 2/200
206/206 [==============================] - 1s 6ms/step - loss: 3.6161
Epoch 3/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5742
Epoch 4/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5778
Epoch 5/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5639
Epoch 6/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5792
Epoch 7/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5529
Epoch 8/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5482
Epoch 9/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5170
Epoch 10/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5628
Epoch 11/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5351
Epoch 12/200
206/206 [==============================] - 1s 4ms/step - loss: 3.5213
Epoch 13/200


1/1 [==============================] - 0s 180ms/step


In [ ]:
predict_dates = pd.date_range(start=pd.to_datetime(stock_train.index[-1]) + pd.Timedelta(days=1), periods=n_output, freq='B')
predictions = pd.Series(predictions[0], index=predict_dates)

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
plt.figure(figsize=(14, 7))
plt.title((codelist[0] + " Forecasted Prices"), fontsize=20)
#Remove [-1006:] to see full view
plt.plot(stock_train[-1006:].index, stock_train[-1006:].values, label="Actual Price")
plt.plot(predictions.index, predictions.values, label="Predicted Price")
plt.plot((stock_test.index), stock_test.values, label="Actual Prices (2024-Present)")
plt.ylabel("Price", fontsize=20)
plt.xlabel("Date", fontsize=20)
plt.legend()


# Save the chart to a file
plt.savefig('LSTM_'+codelist[0]+'_forecast.png')  # The file name can be customized as needed


# Note: If you run this code in a non-GUI environment (such as a server), make sure to set the backend to 'Agg' before
plt.switch_backend('Agg')

# Usually after saving the chart, in order to avoid opening the image window in a non-GUI environment, you can explicitly close the figure
plt.close()


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(predictions, stock_test)
print("LSTM MAE:", mae)

LSTM MAE: 22.205438446580317
